In [2]:
from pathlib import Path
import sys
sys.path.append("../src")
sys.path.append("../src/baseline")


from defs import ROOT, VAL_DATA_PATH, VAL_IMAGES_PATH
from inference.submit import prepare_submit
from inference.grid_search import run_grid_search
from baseline.evaluation.evaluation import calculation_map_050, calculate_meanIOU, competition_metric

import warnings
warnings.filterwarnings("ignore")

In [20]:
exp_names = ["FPN-res101-bs=6-multiscale", 
             "cascade-R50-2fc-FrozenBN-bs=6", 
             "retina-R50-1280-1794-4096-bs=6", 
             "yolov5/yolov5m6-2048-bs=16", 
             "yolov5/yolov5m6-2560-bs=8",
             "yolov5/yolov5l6-1536-smooth=0.1-bs=16",
            ]
exp_dirs = [ROOT / "outputs" / exp_name for exp_name in exp_names]

In [21]:
score_threshes = [0.0001] * len(exp_dirs)
nms_threshes = [0.5, 0.5, 0.5, 0.6, 0.5, 0.5]
tta = True
ensemble_method = "wbf"
ensemble_method_kwargs = {"iou_thr": 0.6}

prepare_submit(VAL_IMAGES_PATH, Path("."), exp_dirs, score_threshes, nms_threshes, tta=tta, ensemble_method=ensemble_method, **ensemble_method_kwargs)
competition_metric(str(VAL_DATA_PATH / "coco.json"), "detection_predictions.json",
                   str(VAL_DATA_PATH / "segmentation_gt.json"), "segmentation_predictions.json")

PANOPTIC_FPN.COMBINED.ENABLED is no longer used.  model.inference(do_postprocess=) should be used to toggle postprocessing.
PANOPTIC_FPN.COMBINED.ENABLED is no longer used.  model.inference(do_postprocess=) should be used to toggle postprocessing.


Unable perform a TTA for meta arch RetinaNet


segmentation submit preparation: 100%|██████████| 411/411 [00:00<00:00, 183350.24it/s]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=2.25s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=12.93s).
Accumulating evaluation results...
DONE (t=2.25s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.347
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.675
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.297
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.282
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.394
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.428
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.265
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.507
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet

100%|██████████| 411/411 [00:24<00:00, 16.75it/s]

Detection: 0.6745442806283277
Segmentation: 0.887531122109563


0.73844

## Grid search

In [9]:
exp_names = ["yolov5/yolov5m6-2048-bs=16", "yolov5/yolov5m6-2560-bs=8", "yolov5/yolov5l6-1536-smooth=0.1-bs=16"]
exp_dirs = [ROOT / "outputs" / exp_name for exp_name in exp_names]

In [12]:
score_threshes = [[0.0001] * len(exp_dirs)]
nms_threshes = [[0.6, 0.5, 0.5]]

ensemble_methods = ["wbf"]
iou_threshes = [0.6]
weights = [[1, 1, 1]]
#ensemble_methods_kwargs = [{"iou_thr": 0.5, "weights": [1, 1, 1, 1]}]
ensemble_methods_kwargs = [{"iou_thr": iou_thr, "weights": w} for iou_thr in iou_threshes for w in weights]
metrics = run_grid_search(VAL_IMAGES_PATH, Path("."), exp_dirs, score_threshes, nms_threshes, ensemble_methods, ensemble_methods_kwargs)

100%|██████████| 411/411 [08:13<00:00,  1.20s/it]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.91s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=9.34s).
Accumulating evaluation results...
DONE (t=1.29s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.343
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.647
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.300
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.281
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.383
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.437
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.261
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.492
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets